In [ ]:
import matplotlib
matplotlib.use('nbagg')
print(matplotlib.get_backend())

import os
import matplotlib.pyplot as plt
from os import listdir
import random
import numpy as np
import cv2 as cv

In [ ]:
#loading dataset 
dataset_location = 'C:/Users/sushl/Downloads/Visual-Odometry-master/Visual-Odometry-master/KITTI_sample/images'
#dataset_location = 'C:/Users/sushl/Downloads/Archives/data_odometry_gray/dataset/sequences/07/image_0'
#dataset_location = 'C:/Users/sushl/Desktop/visual odometry/adc'
L = os.listdir(dataset_location)
L.sort()

In [ ]:
#ground Truth
ground_truth = np.loadtxt('C:/Users/sushl/Downloads/Visual-Odometry-master/Visual-Odometry-master/KITTI_sample/poses.txt',delimiter = ' ')
#ground_truth = np.loadtxt('C:/Users/sushl/Downloads/Archives/data_odometry_poses/dataset/poses/07.txt',delimiter = ' ')
ground_truth.shape
gx = ground_truth[:400,3]
gz = ground_truth[:400,11]

In [ ]:
#calibration matrix

k =np.array([[7.188560000000e+02, 0.000000000000e+00, 6.071928000000e+02], 
             [0.000000000000e+00, 7.188560000000e+02, 1.852157000000e+02], 
             [0.000000000000e+00, 0.000000000000e+00, 1.000000000000e+00]])

"""
k = np.array([[518.56666108, 0., 329.45801792],
    [0., 518.80466479, 237.05589955],
    [  0., 0., 1.]])
"""
sift = cv.SIFT_create()
bf = cv.BFMatcher()
brief = cv.xfeatures2d.BriefDescriptorExtractor_create()


In [ ]:
def keypoints(img,Threshold=75):
    fast = cv.FastFeatureDetector_create()
    fast.setNonmaxSuppression(1)
    fast.setThreshold(Threshold)
    kp = fast.detect(img,None)
    pt = np.float32([ kp[m].pt for m in range(len(kp))]).reshape(-1,1,2)
    return kp,pt 

In [ ]:
"""
def keypoints(img):
    corners = cv.goodFeaturesToTrack(img, mask = None, maxCorners = 800, qualityLevel = 0.01, minDistance = 1, blockSize = 5, useHarrisDetector = True, k = 0.04  )
    #print(corners.shape)
    #print(corners.shape,corners)
    return corners,corners
"""

In [ ]:
def optical_flow_matches(img,trig_frame,pt1,window_size):
    # Parameters for lucas kanade optical flow
    lk_params = dict( winSize  = (window_size,window_size),
                      maxLevel = 2,
                      criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
    ptimg, st, err = cv.calcOpticalFlowPyrLK(trig_frame, img, pt1, None, **lk_params)

    pt1i, st, err = cv.calcOpticalFlowPyrLK(img, trig_frame, ptimg, None, **lk_params)
    return ptimg,pt1i

In [ ]:
def point3D(k,R,t,pts1,pts2):
    rt = np.zeros((3,4))
    rt[:3,:3] = np.identity(3)
    projMatr1 = k@rt
    rt2 = np.zeros((3,4))
    rt2[:3,:3] = R
    rt2[:,3] = t.reshape((3))
    projMatr2 = k@rt2
    points4D = cv.triangulatePoints(projMatr1,projMatr2,pts1 ,pts2)
    points3D = points4D / points4D[3,:]
    return points3D.T[:,:3]

In [ ]:
"""
#for first two images
img1 = cv.imread(dataset_location +'/'+ L[0],cv.IMREAD_GRAYSCALE)
img2 = cv.imread(dataset_location +'/'+ L[1],cv.IMREAD_GRAYSCALE)
pts,kp1= keypoints(img1)
kp2,kp1 = optical_flow_matches(img2,img1,kp1,10)

E, mask = cv.findEssentialMat(kp1,kp2,k,cv.RANSAC, prob = 0.999,threshold = 0.4,mask=None)
#inlier points
pts1 = kp1[mask.ravel()==1]
pts2 = kp2[mask.ravel()==1]
matches = {}
#print(tuple(kp1[0].flatten()))
for i in range(len(pts2)):
    matches[tuple(pts1[i].flatten())] = tuple(pts2[i].flatten())
print(matches)
#recovering pose info
retval, R, t, mask = cv.recoverPose(E, pts1, pts2, k)
#print(t)
pointcloud = point3D(k,R,t,pts1,pts2)
retval, rvec, t, inliers = cv.solvePnPRansac(pointcloud,pts2, k, (0,0,0,0),useExtrinsicGuess = True ,iterationsCount = 100,reprojectionError = 8.0,confidence = 0.90,flags = 1)
R,Jec = cv.Rodrigues(rvec)
print(t,len(inliers))
translations = []
rotations = []
for i in range(3,15):
    img3 = cv.imread(dataset_location +'/'+ L[i],cv.IMREAD_GRAYSCALE)
    kp3,kp2i = optical_flow_matches(img3,img2,pts2,10)
    #pointcloud = point3D(k,R,t,kp1,kp2)
    matches32 = {}
    for i in range(len(pts2)):
        matches32[tuple(pts2[i].flatten())] = tuple(kp3[i].flatten())
    print(matches32)
    #print(len(kp3),len(kp2))
    retval, rvec, tvec, inliers = cv.solvePnPRansac(pointcloud,kp3, k, (0,0,0,0),useExtrinsicGuess = True ,iterationsCount = 100,reprojectionError = 8.0,confidence = 0.90,flags = 1)
    R3,Jec = cv.Rodrigues(rvec)
    print(tvec,len(inliers))   
    translations.append(tvec)
    rotations.append(R3)

"""


In [ ]:

"""    
        else:
            i1 = cv.imread(dataset_location +'/'+ L[a-1],cv.IMREAD_GRAYSCALE)
            i2 = cv.imread(dataset_location +'/'+ L[a],cv.IMREAD_GRAYSCALE)
            pt,k1= keypoints(img1)
            k2,k1 = optical_flow_matches(i2,i1,k1,15)

            E, mask = cv.findEssentialMat(k1,k2,k,cv.RANSAC, prob = 0.999,threshold = 0.4,mask=None)

            #inlier points
            p1 = k1[mask.ravel()==1]
            p2 = k2[mask.ravel()==1]
            retval, r, ta, mask = cv.recoverPose(E, p1, p2, k)
            translations.append(translations[a-1] + rotations[a-1]@ta)
            #print(len(translations))
            rotations.append(rotations[a-1]@r)
            count = 0
            match = 210
            print('else')
            count = count + 1
            if count >3:
                break
        """

In [ ]:
#for first two images
img1 = cv.imread(dataset_location +'/'+ L[0],cv.IMREAD_GRAYSCALE)
img2 = cv.imread(dataset_location +'/'+ L[1],cv.IMREAD_GRAYSCALE)
pts,kp1= keypoints(img1)
kp2,kp1 = optical_flow_matches(img2,img1,kp1,21)
print(len(kp1))
E, mask = cv.findEssentialMat(kp1,kp2,k,cv.RANSAC, prob = 0.999,threshold = 0.4,mask=None)

#inlier points
pts1 = kp1[mask.ravel()==1]
pts2 = kp2[mask.ravel()==1]

#recovering pose info
retval, R, t, mask = cv.recoverPose(E, pts1, pts2, k)
pointcloud = point3D(k,R,t,pts1,pts2)
retval, rvec, t, inliers = cv.solvePnPRansac(pointcloud,pts2, k, (0,0,0,0),useExtrinsicGuess = True ,iterationsCount = 100,reprojectionError = 8.0,confidence = 0.90,flags = 1)
R,Jec = cv.Rodrigues(rvec)
#set of translations and rotations
translations = []
rotations = []
a = 0
t0 = np.zeros((3,1))
r0 = np.identity(3)

#storing previous imgs
framep1 = img1[:]
framep2 = img2[:]
Rp = R[:]
Tp = t[:]
t0 = np.zeros((3,1))
r0 = np.identity(3)
match = 20000
count = 0
error_set = []
error_set.append(0)
ntcframe = 0
#for remaining images
while(a < 150):#
    print(a)
    img3 = cv.imread(dataset_location +'/'+ L[a],cv.IMREAD_GRAYSCALE)
    kp3,kp2i = optical_flow_matches(img3,img2,pts2,10)
    #match = len(kp3)
    #print(match)
    #count = 0
    #match = threshold for pnp i.e it inliers of previous frame are greater than this threshold we will use pnp directly
    if match > 200: #or error_set[a-1]< 10 :
        retval, rvec, tvec, inliers = cv.solvePnPRansac(pointcloud,kp3, k, (0,0,0,0),useExtrinsicGuess = True ,iterationsCount = 100,reprojectionError = 5.0,confidence = 0.750,flags = 1)
        R3,Jec = cv.Rodrigues(rvec)
        #print(tvec,len(inliers))   
        #translations.append(tvec)
        #rotations.append(R3)
        match = len(inliers)
        #print(match)
        p3d = np.ones((4,pointcloud.shape[0]))
        p3d[:3,:] = pointcloud.T
        projection_mat = np.zeros((3,4))
        projection_mat[:3,:3] = r0@R3
        projection_mat[:,3] = (t0 + r0@tvec).reshape((3))
        proj_pt = k@projection_mat@p3d
        proj_pt = proj_pt.T
        proj_pt= proj_pt / proj_pt[:,2].reshape((-1,1))
        proj_pt = proj_pt[:,:2]
        err = proj_pt - kp3.reshape((-1,2))
        norm_err = np.linalg.norm(err, axis=1)
        error_set.append(np.min(norm_err))
        #print(match,error_set[a])
        if ((a==0 or a==1)or np.linalg.norm(t0 + r0@tvec - translations[a-1-ntcframe])< 6*np.linalg.norm(translations[a-1-ntcframe]-translations[a-2-ntcframe])):
            translations.append(t0 + r0@tvec)
            #print(len(translations))
            rotations.append(r0@R3)
        
        elif np.linalg.norm(t0 + r0@tvec - translations[a-1-ntcframe])> 6*np.linalg.norm(translations[a-1-ntcframe]-translations[a-2-ntcframe]):
            ntcframe = ntcframe + 1
            print('abcd')
        
        print(np.linalg.norm(t0 + r0@tvec - translations[a-1-ntcframe]),translations[a-ntcframe])
        count = 0
        
        
    #else will trigulated point cloud from previous two imgs
    else:
        img1 = cv.imread(dataset_location +'/'+ L[a-2],cv.IMREAD_GRAYSCALE)
        img2 = cv.imread(dataset_location +'/'+ L[a-1],cv.IMREAD_GRAYSCALE)
        pts,kp1= keypoints(img1)
        kp2,kp1 = optical_flow_matches(img2,img1,kp1,15)

        E, mask = cv.findEssentialMat(kp1,kp2,k,cv.RANSAC, prob = 0.999,threshold = 0.4,mask=None)

        #inlier points
        pts1 = kp1[mask.ravel()==1]
        pts2 = kp2[mask.ravel()==1]
        
        r0 = rotations[a-2-ntcframe]
        t0 = translations[a-2-ntcframe]
        retval, R, t, mask = cv.recoverPose(E, pts1, pts2, k)
        pointcloud = point3D(k,R,t,pts1,pts2)
        #error_set.append(2)
        a = a-1
        match = 20000
        print('me')
        count = count + 1
        if count >3:
            break
            
    
    a = a+1

In [ ]:
x0 = []
y0 = []
go = []
g1 = []
for i in range(len(translations)-1):
    y0.append(-1*translations[i][2])
    x0.append(translations[i][0])
    
fig,axes = plt.subplots()
axes.plot(x0, y0,label = 'scale using dist ratio')
axes.plot(gx, gz,color = 'red',label = 'ground_truth')
axes.legend()

plt.show()

In [ ]:
x0 = []
y0 = []
for i in range(150):#len(error_set)-1
    y0.append(error_set[i])
    x0.append(i)
    
fig,axes = plt.subplots()
axes.plot(x0, y0,label = 'error')
axes.legend()

plt.show()

In [ ]:
print(len(translations)-1)

In [ ]:
import numpy as np
import cv2 as cv
import argparse
parser = argparse.ArgumentParser(description='This sample demonstrates Lucas-Kanade Optical Flow calculation. \
                                              The example file can be downloaded from: \
                                              https://www.bogotobogo.com/python/OpenCV_Python/images/mean_shift_tracking/slow_traffic_small.mp4')
parser.add_argument('image', type=str, help='path to image file')
args = parser.parse_args()
cap = cv.VideoCapture(args.image)
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
# Create some random colors
color = np.random.randint(0,255,(100,3))
# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
while(1):
    ret,frame = cap.read()
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # calculate optical flow
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]
    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv.add(frame,mask)
    cv.imshow('frame',img)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)